# Readings of place names with japandata.readings

The maps in `japandata.maps` do not have pronunciation information for the city and prefecture names. 

`japandata.readings` has this information for modern place names. For historical place names I have yet to find a convenient dataset.

In [32]:
from japandata.readings import city_names, pref_names

In [33]:
city_names.head(5)

,prefecture,city,prefecture-kana,city-kana,code,prefecture-romaji,city-romaji
0,北海道,札幌市,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼ,01100,hokkaidou,sapporo
1,北海道,函館市,ﾎｯｶｲﾄﾞｳ,ﾊｺﾀﾞﾃｼ,01202,hokkaidou,hakodate
2,北海道,小樽市,ﾎｯｶｲﾄﾞｳ,ｵﾀﾙｼ,01203,hokkaidou,otaru
3,北海道,旭川市,ﾎｯｶｲﾄﾞｳ,ｱｻﾋｶﾜｼ,01204,hokkaidou,asahikawa
4,北海道,室蘭市,ﾎｯｶｲﾄﾞｳ,ﾑﾛﾗﾝｼ,01205,hokkaidou,muroran


In [34]:
pref_names.head(5)

,prefecture,prefecture-kana,code,prefecture-romaji
0,北海道,ﾎｯｶｲﾄﾞｳ,01,hokkaidou
1,青森県,ｱｵﾓﾘｹﾝ,02,aomori
2,岩手県,ｲﾜﾃｹﾝ,03,iwate
3,宮城県,ﾐﾔｷﾞｹﾝ,04,miyagi
4,秋田県,ｱｷﾀｹﾝ,05,akita


## How to combine with japandata.maps

In [35]:
from japandata.readings import city_names, pref_names
from japandata.maps import load_map
pref_map = load_map(date=2022, scale="jp_pref", quality="coarse")
city_map = load_map(date=2022, scale="jp_city_dc", quality="coarse")

In [36]:
pref_map_with_readings = pref_map.merge(pref_names, how='left', on=['code','prefecture'])

In [37]:
pref_map_with_readings.head(5)

,prefecture,code,geometry,prefecture-kana,prefecture-romaji
0,北海道,01,"MULTIPOLYGON (((145.29132 43.38680, 145.29812 ...",ﾎｯｶｲﾄﾞｳ,hokkaidou
1,青森県,02,"MULTIPOLYGON (((141.58198 40.53788, 141.58179 ...",ｱｵﾓﾘｹﾝ,aomori
2,岩手県,03,"MULTIPOLYGON (((142.00416 39.75169, 141.98959 ...",ｲﾜﾃｹﾝ,iwate
3,宮城県,04,"MULTIPOLYGON (((140.93102 37.89616, 140.92789 ...",ﾐﾔｷﾞｹﾝ,miyagi
4,秋田県,05,"MULTIPOLYGON (((140.72954 39.56492, 140.72898 ...",ｱｷﾀｹﾝ,akita


In [38]:
city_map_with_readings = city_map.merge(city_names, how='left', on=['prefecture','city'], suffixes=('','_merge')).drop('code_merge',axis=1)

In [39]:
city_map_with_readings.head(5)

,prefecture,bureau,county,city,code,special,geometry,prefecture-kana,city-kana,prefecture-romaji,city-romaji
0,北海道,石狩振興局,札幌市,札幌市,01100,designated-city,"MULTIPOLYGON (((141.50543 43.02086, 141.48711 ...",ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼ,hokkaidou,sapporo
1,北海道,渡島総合振興局,NaN,函館市,01202,NaN,"MULTIPOLYGON (((140.70964 41.80850, 140.71957 ...",ﾎｯｶｲﾄﾞｳ,ﾊｺﾀﾞﾃｼ,hokkaidou,hakodate
2,北海道,後志総合振興局,NaN,小樽市,01203,NaN,"MULTIPOLYGON (((141.29184 43.18504, 141.27035 ...",ﾎｯｶｲﾄﾞｳ,ｵﾀﾙｼ,hokkaidou,otaru
3,北海道,胆振総合振興局,NaN,室蘭市,01205,NaN,"MULTIPOLYGON (((141.05440 42.35211, 141.04496 ...",ﾎｯｶｲﾄﾞｳ,ﾑﾛﾗﾝｼ,hokkaidou,muroran
4,北海道,渡島総合振興局,松前郡,松前町,01331,NaN,"MULTIPOLYGON (((139.80370 41.36571, 139.81547 ...",ﾎｯｶｲﾄﾞｳ,ﾏﾂﾏｴﾁｮｳ,hokkaidou,matsumae


In [40]:
pref_map_with_readings['prefecture-romaji']=pref_map_with_readings['prefecture-romaji'].str.replace('ou','ō').str.replace('oo','ō').str.title()

In [41]:
import os
import matplotlib.pyplot as plt
from japandata.maps import load_map

PLOT_FOLDER = "./readings/"
os.makedirs(PLOT_FOLDER, exist_ok=True)

import folium
from branca.colormap import linear
from folium.features import GeoJsonTooltip

unhighlighted_style = {
    "color": "black",
    "weight": 1,
    "fillOpacity": 0.2,
}

highlighted_style = unhighlighted_style | {"weight": 4}

map_style = {"location": [35.67, 139], "zoom_start": 5, "tiles": "None", "attr": " "}

m = folium.Map(**map_style)

tooltip = GeoJsonTooltip(
    fields=["prefecture-romaji", "code"],
    aliases=["Prefecture:", "Code:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

folium.GeoJson(
    pref_map_with_readings,
    style_function=lambda feature: unhighlighted_style,
    highlight_function=lambda feature: highlighted_style,
    zoom_on_click=True,
    tooltip=tooltip,
).add_to(m)

m.save(PLOT_FOLDER + "pref_with_readings_interactive.html")

m